### Tone 산출

In [27]:
import pandas as pd
import os

lexicon_path = r"C:\KDT\bok_pjt\team2_forecast_pjt\db\lexicon\total_lexicon.csv"
total_lexicon = pd.read_csv(lexicon_path, encoding='utf-8-sig', index_col=0)

hawkish_set = set(total_lexicon[total_lexicon['label'] == 'hawkish'].index)
dovish_set = set(total_lexicon[total_lexicon['label'] == 'dovish'].index)

print(f"✅ 사전 로드 완료: 매파 단어 {len(hawkish_set)}개, 비둘기파 단어 {len(dovish_set)}개")

✅ 사전 로드 완료: 매파 단어 9264개, 비둘기파 단어 8522개


In [28]:
import numpy as np
from datetime import timedelta
import sys
!{sys.executable} -m pip install pyarrow

import pandas as pd

df= pd.read_parquet(r"C:\KDT\bok_pjt\team2_forecast_pjt\db\tone\df_for_tone_260107.parquet")

In [29]:
# tokens 리스트화
import ast
def convert_to_list(x):
    if isinstance(x, str): 
        return ast.literal_eval(x) 
    return x

df['tokens'] = df['tokens'].apply(convert_to_list)
print(f"token data 타입: {type(df['tokens'].iloc[0])}")


token data 타입: <class 'numpy.ndarray'>


### 뉴스, 채권보고서 기반 daily score 산출

In [30]:
# 문장
def calculate_tone(n_hawkish, n_dovish):
    denominator = n_hawkish + n_dovish
    if denominator == 0:
        return 0 
    return (n_hawkish - n_dovish) / denominator

df_tone = df.copy()
df_tone = df_tone[df_tone['category'] != '의사록'].reset_index(drop=True).copy()
df_tone = df_tone[df_tone['category'] != 'press'].reset_index(drop=True).copy()

df_tone


,doc_id,sent_id,date,content,tokens,category,source,date_1m,rate_today,rate_1m,diff,label
0,237430,4,2012-01-01,하지만 시장 전문가들은 내년 중순 이후 금값이 올해 최고치인 온스당 1920달러를 ...,[하/VV;있/VV;보/VV],뉴스,한국경제,2012-01-31,3.29,3.27,-0.02,neutral
1,237283,17,2012-01-01,국가채무위기란 국가가 빌린 돈을 제때 갚지 못해 기업처럼 부도가 날 수 있는 상황을...,[날/VV;있/VV],뉴스,한국경제,2012-01-31,3.29,3.27,-0.02,neutral
2,237283,35,2012-01-01,그리스가 유로에 가입하지 않았다면 그랬다면 그리스와 독일 간의 환율이 변동하면서 경...,[있/VV],뉴스,한국경제,2012-01-31,3.29,3.27,-0.02,neutral
3,237283,7,2012-01-01,3년 전에 비해 유로화 가치가 원화 대비 많이 떨어졌거든요.,[많이/MAG;떨어/VV],뉴스,한국경제,2012-01-31,3.29,3.27,-0.02,neutral
4,237283,28,2012-01-01,여행객도 크게 늘고 덩달아 각국의 여행 수입도 늘어났죠.,[늘/VV],뉴스,한국경제,2012-01-31,3.29,3.27,-0.02,neutral
...,...,...,...,...,...,...,...,...,...,...,...,...
2093435,158861,38,2025-12-30,막대한 국채를 보유한 일본이 곧 금리를 올릴 수 밖에 없을 것으로 보고 공매도에 들...,[많/VA],뉴스,한국경제,2026-01-29,2.54,NaN,NaN,neutral
2093436,158861,22,2025-12-30,그 달에 비트코인과 금은 모두 사상 최고치를 경신했다.,[최고/NNG;경신/NNG;하/VV],뉴스,한국경제,2026-01-29,2.54,NaN,NaN,neutral
2093437,1621,15,2025-12-30,AI 버블이 곧 터질지에 대한 질문에는 버블이 있지만 관리 가능한 수준 이라는 응답...,[가장/MAG;많/VA],뉴스,매일경제,2026-01-29,2.54,NaN,NaN,neutral
2093438,158861,13,2025-12-30,2008년 서브프라임 모기지 거품을 예언해 유명해진 마이클 버리가 운영하는 사이언 ...,[공시/NNG;하/VV],뉴스,한국경제,2026-01-29,2.54,NaN,NaN,neutral


In [31]:
# 1. 매파 단어 개수 세기
df_tone.loc[:, 'n_h_feat'] = df_tone['tokens'].apply(lambda x: len([w for w in x if w in hawkish_set]) if x is not None else 0)

# 2. 비둘기파 단어 개수 세기
df_tone.loc[:, 'n_d_feat'] = df_tone['tokens'].apply(lambda x: len([w for w in x if w in dovish_set]) if x is not None else 0)

# 3. 톤 지수 계산
df_tone.loc[:, 'tone_s'] = df_tone.apply(lambda row: calculate_tone(row['n_h_feat'], row['n_d_feat']), axis=1)

# 4. 성향 분류
df_tone.loc[:, 'is_h_sent'] = df_tone['tone_s'] > 0
df_tone.loc[:, 'is_d_sent'] = df_tone['tone_s'] < 0

In [32]:
# 문서
# 문서별로 매파 문장 수와 비둘기파 문장 수 sum
doc_level = df_tone.groupby(['doc_id', 'date']).agg(
    n_h_sents=('is_h_sent', 'sum'),
    n_d_sents=('is_d_sent', 'sum')
).reset_index()

doc_level['tone_i'] = doc_level.apply(lambda row: calculate_tone(row['n_h_sents'], row['n_d_sents']), axis=1)

In [33]:
# 날짜별 지수 계산
daily_tone = doc_level.groupby('date')['tone_i'].mean().reset_index()
daily_tone.columns = ['date', 'z_newsbonds']
daily_tone = daily_tone.sort_values(by='date', ascending=True).reset_index(drop=True)
display(daily_tone.head())

,date,z_newsbonds
0,2012-01-01,-0.273077
1,2012-01-02,-0.385376
2,2012-01-03,-0.337913
3,2012-01-04,-0.229573
4,2012-01-05,-0.476000


In [34]:
daily_tone.to_csv('daily_tone_news_bonds.csv', index=False, encoding='utf-8-sig')

### 뉴스, 채권 daily tone 점수 월별 tone 점수로 변환 (평균)

In [44]:
import pandas as pd

monthly_newsbonds = daily_tone.resample('MS', on='date')['z_newsbonds'].mean().reset_index()
monthly_newsbonds.columns = ['date', 'z_newsbonds']

In [45]:
monthly_newsbonds

,date,z_newsbonds
0,2012-01-01,-0.308572
1,2012-02-01,-0.314463
2,2012-03-01,-0.306507
3,2012-04-01,-0.370363
4,2012-05-01,-0.441889
...,...,...
163,2025-08-01,0.246915
164,2025-09-01,0.189342
165,2025-10-01,0.266826
166,2025-11-01,0.235223


### 의사록 월별 tone 산출

In [35]:
df_meeting_tone = df.copy()
df_meeting_tone = df_meeting_tone[df_meeting_tone['category'] == '의사록'].copy()

In [36]:
!pip install openpyxl

In [37]:
date_mapping = pd.read_excel(r'C:\KDT\bok_pjt\team2_forecast_pjt\db\tone\meeting_date_change.xlsx')

date_mapping['회의 날짜'] = pd.to_datetime(date_mapping['회의 날짜'])
date_mapping['업로드 날짜'] = pd.to_datetime(date_mapping['업로드 날짜'])
df_meeting_tone['date'] = pd.to_datetime(df_meeting_tone['date'])

df_meeting_tone = pd.merge(df_meeting_tone, date_mapping, 
                           left_on='date', right_on='회의 날짜', 
                           how='left')

df_meeting_tone['date'] = df_meeting_tone['업로드 날짜'].fillna(df_meeting_tone['date'])
df_meeting_tone = df_meeting_tone.drop(columns=['회의 날짜', '업로드 날짜'])

df_meeting_tone

,doc_id,sent_id,date,content,tokens,category,source,date_1m,rate_today,rate_1m,diff,label
0,237471,19,2012-02-28,이상과 같은 의견교환을 바탕으로 통화정책방향 에 관해 다음과 같은 토론 이 있었음,"[같/VA;의견/NNG;교환/NNG;바탕/NNG;통화정책/NNG, 방향/NNG;관하...",의사록,한국은행,2012-02-12,3.250,3.250,0.000,neutral
1,237471,18,2012-02-28,최근 들어서 외국인 투자자금 유출이 빈번히 발생하고 있 음에도 불구하고 장기금리가 ...,[있/VA;의견/NNG;나타나/VV],의사록,한국은행,2012-02-12,3.250,3.250,0.000,neutral
2,237471,21,2012-02-28,금융완화의 정도를 추정할 때 균형실질금리의 수준이 낮게 추정될 경우 금융완화의 정도...,[있/VA;의견/NNG;제시/NNG],의사록,한국은행,2012-02-12,3.250,3.250,0.000,neutral
3,237472,13,2012-03-27,최근 정부의 30년물 장기 국고채 발행 확대가 통화정책 에 미치는 영향에 대해 물었...,[나타나/VV],의사록,한국은행,2012-03-10,3.250,3.250,0.000,neutral
4,237472,8,2012-03-27,최근 유입되는 외화자금 은 단기적인 환차익이나 재정거래 유인에 의한 것일 수 있으며...,[있/VA;의견/NNG;나타나/VV],의사록,한국은행,2012-03-10,3.250,3.250,0.000,neutral
...,...,...,...,...,...,...,...,...,...,...,...,...
4316,237620,7,2025-12-16,또한 다른 최근 크게 늘어난 등의 수익증권을 2 지표에 서 제외할 경우 현재 대인 ...,[필요/NNG;있/VA],의사록,한국은행,2025-12-27,2.513,2.522,0.009,neutral
4317,237620,9,2025-12-16,이에 대해 관련 부서 최근 시장금리 및 원 달러 환율 상승 등의 영향으로 금 융시장...,[되/VV],의사록,한국은행,2025-12-27,2.513,2.522,0.009,neutral
4318,237620,5,2025-12-16,이와 관련하여 유동성 확대가 자산가격 상승의 원인이기도 하지 만 경제주체들의 신용 ...,[있/VA;의견/NNG;나타나/VV],의사록,한국은행,2025-12-27,2.513,2.522,0.009,neutral
4319,237620,1,2025-12-16,의장이 전일 보고내용을 접수하고 한국은행법 제28조에 따라 통화정책방향 을 상정하였...,"[내용/NNG;접수/NNG;조/NNG;통화정책/NNG;방향/NNG, 상정/NNG]",의사록,한국은행,2025-12-27,2.513,2.522,0.009,neutral


In [38]:
# 1. 매파 단어 개수 세기
df_meeting_tone.loc[:, 'n_h_feat'] = df_meeting_tone['tokens'].apply(lambda x: len([w for w in x if w in hawkish_set]) if x is not None else 0)

# 2. 비둘기파 단어 개수 세기
df_meeting_tone.loc[:, 'n_d_feat'] = df_meeting_tone['tokens'].apply(lambda x: len([w for w in x if w in dovish_set]) if x is not None else 0)

# 3. 톤 지수 계산
df_meeting_tone.loc[:, 'tone_s'] = df_meeting_tone.apply(lambda row: calculate_tone(row['n_h_feat'], row['n_d_feat']), axis=1)

# 4. 성향 분류
df_meeting_tone.loc[:, 'is_h_sent'] = df_meeting_tone['tone_s'] > 0
df_meeting_tone.loc[:, 'is_d_sent'] = df_meeting_tone['tone_s'] < 0

In [39]:
# 문서
# 문서별로 매파 문장 수와 비둘기파 문장 수 sum
doc_meeting_level = df_meeting_tone.groupby(['doc_id', 'date']).agg(
    n_h_sents=('is_h_sent', 'sum'),
    n_d_sents=('is_d_sent', 'sum')
).reset_index()

doc_meeting_level['tone_i'] = doc_meeting_level.apply(lambda row: calculate_tone(row['n_h_sents'], row['n_d_sents']), axis=1)

In [40]:
doc_meeting_level

,doc_id,date,n_h_sents,n_d_sents,tone_i
0,237471,2012-02-28,0,1,-1.0
1,237472,2012-03-27,0,0,0.0
2,237473,2012-04-24,1,1,0.0
3,237474,2012-05-29,1,0,1.0
4,237475,2012-06-26,4,0,1.0
...,...,...,...,...,...
144,237616,2025-07-15,0,0,0.0
145,237617,2025-07-29,1,1,0.0
146,237618,2025-09-16,0,1,-1.0
147,237619,2025-11-11,0,2,-1.0


In [41]:
# 날짜별 지수 계산
monthly_minutes_tone = doc_meeting_level.resample('MS', on='date')['tone_i'].mean().reset_index()

monthly_minutes_tone.columns = ['date', 'z_min']

display(monthly_minutes_tone.head())

,date,z_min
0,2012-02-01,-1.0
1,2012-03-01,0.0
2,2012-04-01,0.0
3,2012-05-01,1.0
4,2012-06-01,1.0


In [42]:
monthly_minutes_tone.to_csv('monthly_minutes_tone.csv', index=False, encoding='utf-8-sig')